In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (GCNConv, GATConv, GINConv, global_mean_pool, 
                                global_max_pool, global_add_pool, MessagePassing)
from torch_geometric.data import Data, DataLoader
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d, LayerNorm
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import depolarizing_error, NoiseModel
import pandas as pd
from typing import List, Tuple
from qiskit_ibm_runtime import Sampler, Session, Options

def pauli_noise_model(p1=0.01, p2=0.1):
    nm = NoiseModel()
    one_q_gates = ["h","s","t","x","y","z","rz","sx"]
    two_q_gates = ["cx","cz"]
    one_q_err = depolarizing_error(p1, 1)
    two_q_err = depolarizing_error(p2, 2)
    nm.add_all_qubit_quantum_error(one_q_err, one_q_gates)
    nm.add_all_qubit_quantum_error(two_q_err, two_q_gates)
    return nm

def get_ideal_distribution(circuit, shots=8192):
    """Get ideal (noiseless) distribution from circuit."""
    sim = AerSimulator()
    full_circuit = circuit.copy()
    full_circuit.measure_all()
    
    result = sim.run(full_circuit, shots=shots).result()
    counts = result.get_counts()
    
    n = circuit.num_qubits
    dist = np.zeros(2**n, dtype=np.float32)
    for bitstring, c in counts.items():
        idx = int(bitstring, 2)
        dist[idx] = c / shots
    
    return dist

def random_STHCZ_circuit(n_qubits: int, depth_layers: int, p_cz=0.5):
    qc = QuantumCircuit(n_qubits)
    oneq_gates = ["h","s","t"]
    for _ in range(depth_layers):
        for q in range(n_qubits):
            g = np.random.choice(oneq_gates)
            getattr(qc, g)(q)
        if np.random.rand() < p_cz and n_qubits >= 2:
            a, b = np.random.choice(n_qubits, size=2, replace=False)
            qc.cz(a, b)
        qc.barrier()
    return qc

In [2]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeAthensV2

backend=FakeAthensV2()
print(backend)
noisy_sampler=SamplerV2(backend)

In [3]:
circ = random_STHCZ_circuit(3,5) # Build random circuit of 3 qubits and 5 layers

circ.measure_all()

# Transpile circuit
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
circ = transpile(
                circ, 
                backend,
                optimization_level=0  # No optimization to preserve structure
            )


In [4]:
noisy_sampler.options.twirling.enable_measure=True
noisy_sampler.options.twirling.shots_per_randomization= "auto"
noisy_sampler.options.twirling.strategy= "active-circuit"
noisy_sampler.options.twirling.enable_gates=True

In [5]:
result_noisy=noisy_sampler.run([circ], shots=1024).result()

C:\Users\zacha\anaconda3\envs\tf\Lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:269: UserWarning: Options {'twirling': {'enable_gates': True, 'enable_measure': True, 'shots_per_randomization': 'auto', 'strategy': 'active-circuit'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [ ]:
import warnings
from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeAthensV2

# --- Step 1: Get the Noise Model Locally ---
# We still use FakeAthensV2, but just to grab its noise model.
backend = FakeAthensV2()
noise_model = backend.target.noise_model

# --- Step 2: Build and Transpile Your Circuit ---
circ = random_circuit(3, 5, measure=False)
circ.measure_all() 

circ_transpiled = transpile(
    circ, 
    backend,
    optimization_level=0
)

# --- Step 3: Connect to the (Free) Cloud Service ---
# This will look for your credentials, or prompt you to save them.
# You can create a free account at https://quantum.ibm.com/
try:
    service = QiskitRuntimeService()
except Exception:
    # Run this once to save your API key
    # QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN")
    # service = QiskitRuntimeService()
    print("Please save your IBM Quantum API token first.")
    # You'll need to handle this login part
    raise

# --- Step 4: Configure the Sampler to use the Cloud Simulator ---
# We tell the Sampler to use the real service and the cloud simulator
sampler = SamplerV2(
    service=service,
    backend="ibm_qasm_simulator", # Use the cloud simulator
)

# --- Step 5: Set BOTH Twirling and Simulator Options ---
# 5a. Set your twirling options (The cloud service will use these)
sampler.options.twirling.enable_gates = True
sampler.options.twirling.enable_measure = True
sampler.options.twirling.strategy = "active-circuit"

# 5b. Set the simulator options (The simulator will use these)
# This is the key: we "upload" the noise model
sampler.options.simulator.noise_model = noise_model
sampler.options.simulator.basis_gates = basis_gates
sampler.options.simulator.coupling_map = backend.target.coupling_map

# --- Step 6: Run the Job ---
# This job is sent to the cloud, where twirling is applied,
# and then executed on the simulator with the Athens noise.
print("Running job on cloud simulator with local noise model and twirling...")
with warnings.catch_warnings():
    # Suppress warnings about running on a simulator
    warnings.simplefilter("ignore", category=UserWarning)
    
    job = sampler.run([circ_transpiled], shots=1024)
    result_noisy = job.result()

# 7. Get and print the output distribution
quasi_dist = result_noisy[0].data.c.quasi_dists[0]
print("Noisy + Twirled output distribution:")
print(quasi_dist)